In [ ]:
import importlib

package_name = "datasets"

try:
    importlib.import_module(package_name)
    print(f"{package_name} is already installed.")
except ImportError:
    print(f"{package_name} is not installed. Installing it now...")
    !pip install {package_name}
    print(f"{package_name} has been successfully installed.")

datasets is not installed. Installing it now...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
datasets has been successfully installed.


In [ ]:
import importlib

package_name = "accelerate"

try:
    importlib.import_module(package_name)
    print(f"{package_name} is already installed.")
except ImportError:
    print(f"{package_name} is not installed. Installing it now...")
    !pip install {package_name}
    print(f"{package_name} has been successfully installed.")

accelerate is not installed. Installing it now...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.3 MB/s eta 0:00:00
accelerate has been successfully installed.


In [ ]:
import os
import shutil
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import classification_report


from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoModelForPreTraining
from transformers import BertTokenizer, AutoTokenizer, DataCollatorWithPadding, BertForSequenceClassification

from datasets import load_metric
from datasets import load_dataset
from datasets import load_from_disk

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

%cd "/content/drive/MyDrive/6386"

# from colab.file_utils import load_required
# load_required(force_reinstall_pytorch=True)

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/12WI-y1mku9myCGqwJZuSs-bf71Ygc7th/6386


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
print(os.getcwd())

/content/drive/.shortcut-targets-by-id/12WI-y1mku9myCGqwJZuSs-bf71Ygc7th/6386


In [ ]:
saved_model_dir = "saved_model"

# Check if the saved_model_dir exists
if os.path.exists(saved_model_dir):
    # If it exists, remove its contents recursively
    shutil.rmtree(saved_model_dir)

In [ ]:
# !pip install datasets

In [ ]:
# # Hugging Face pre-trained
# tokenizer = AutoTokenizer.from_pretrained("nlpaueb/sec-bert-base")
# model = AutoModelForPreTraining.from_pretrained("nlpaueb/sec-bert-base")

In [ ]:
# checkpoint = "bert-base-uncased"
checkpoint = "bert-base-cased"
# checkpoint = "distilbert-base-uncased"
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
# raw_datasets = load_from_disk("finetune_data_trial")
raw_datasets = load_from_disk("finetune_data")

def tokenize_function(example):
    return tokenizer(example["text"], padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # pad all the examples to the length of the longest element when we batch elements together — dynamic padding.

Map:   0%|          | 0/8925 [00:00<?, ? examples/s]

Map:   0%|          | 0/2232 [00:00<?, ? examples/s]

Map:   0%|          | 0/876 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8925
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2232
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 876
    })
    test_unlabeled: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 80793
    })
})

In [ ]:
tokenized_datasets['train']['text'][:10]

['$AAPL wants lower. up waves getting smaller on the 2 min',
 '$GOOGL gets an upgrade price target $970',
 "Watch NKD here. Selling volume has been picking up and it's now below its 20 day moving average. Candle still needs to close.",
 "$FB's stock experiences volatility as the company grapples with controversies related to content moderation and user data privacy.",
 "Doubts about $AAPL's direction are intensifying. Recent decisions are questionable. ????",
 'Green Weekly Triangle on MNK,....Open ong position at 3.17,...Opening both ong and Short   ',
 "QCOM looking good here for a long, yesterday's close as stop",
 'AAP moving down with volume. Couldnt cope with the descending trend line. Trend is still down. Perhaps 460 is not impossible after all.',
 'WFM - looks like a bear flag, we are short, will respect the flag and exit on a close above   ',
 'Costco: A Premier Retail Dividend Play https://t.co/Fa5cnh2t0t $COST']

In [ ]:
metric = load_metric("glue", "mrpc")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-14-093e3287862a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")


In [ ]:
# !pip install transformers[torch]
# !pip install accelerate -U

In [ ]:
import accelerate
print(accelerate.__version__)

0.24.1


In [ ]:
training_args = TrainingArguments(
    output_dir="test-trainer",
    evaluation_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=256,
    # During the first 500 training steps, the learning rate gradually increases from 0 (or a small base rate) to the specified learning rate.
    # This gradual increase helps in stabilizing the training process and often leads to better performance, as it prevents the model from making too large updates too quickly.
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir='logs',
    logging_steps=10,  # how frequently the training progress is logged
    save_strategy="epoch",  # Set save strategy to "epoch" to save both best and last models
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",  # disable wandb
    learning_rate=1e-4,
    # fp16=True,  # Enable mixed precision training
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.487900,0.373879,0.840502,0.863392
2,0.307500,0.444309,0.828405,0.865093
3,0.298900,0.429997,0.859767,0.881394
4,0.273700,0.488690,0.863351,0.885981
5,0.260100,0.491391,0.863351,0.882828
6,0.124300,0.482482,0.861111,0.876984
7,0.128200,0.540972,0.859767,0.876723
8,0.102200,0.642647,0.855287,0.867460
9,0.038000,0.670001,0.867832,0.885615
10,0.168500,0.682500,0.866487,0.883229


TrainOutput(global_step=5580, training_loss=0.20960469309734614, metrics={'train_runtime': 795.3967, 'train_samples_per_second': 112.208, 'train_steps_per_second': 7.015, 'total_flos': 6215756641264620.0, 'train_loss': 0.20960469309734614, 'epoch': 10.0})

In [ ]:
output_dir = "saved_model"
os.makedirs(output_dir, exist_ok=True)

# Save the best model and the last model
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.txt',
 'saved_model/added_tokens.json')

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(876, 2) (876,)


In [ ]:
predictions

PredictionOutput(predictions=array([[ 2.9296482, -2.4611754],
       [ 2.930862 , -2.464403 ],
       [ 1.5338113, -2.3713057],
       ...,
       [ 2.9295638, -2.4609487],
       [-1.5164928,  2.3670745],
       [ 2.9285212, -2.4581392]], dtype=float32), label_ids=array([1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
    

In [ ]:
probabilities = softmax(predictions.predictions, axis=1)
predicted_labels = np.argmax(probabilities, axis=1)

print("Probabilities:\n", probabilities)
print("Predicted Labels:\n", predicted_labels)

Probabilities:
 [[0.9954625  0.00453754]
 [0.99548244 0.00451752]
 [0.98025894 0.01974104]
 ...
 [0.99546105 0.00453894]
 [0.0201624  0.9798376 ]
 [0.9954436  0.00455638]]
Predicted Labels:
 [0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 0 1 1 1 0 0 1
 1 1 0 0 1 1 1 1 0 1 0 1 1 1 0 0 0 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1
 0 1 0 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1 1 0 0 0 1
 1 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 0 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 1 0 1 1 0 0
 0 0 1 0 1 1 1 1 1 0 1 0 0 0 1 0 1 1 1 1 1 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0
 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 1 1 0 1 1 1
 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 0 0 0 1 1 0 1 1 1 1 1 0 0 1 0 1 1 1 0 1
 1 1 0 1 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 1 0 0 0 1 1 0 1 1 0 1 1 1 0 1 1 1 1
 0 0 0 1 0 1 1 0 0 1 1 1 0 1 1 1 0 0 1 1 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1
 0 1 1 0 0 1 1 1 0 1 0 0 1 0 1 0 1 1 1 0 0 1 1 1 0 1 1 1 0 

In [ ]:
true_labels = tokenized_datasets["test"]["label"]

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.7579908675799086
Precision: 0.7651006711409396
Recall: 0.7579908675799086
F1 Score: 0.7522090232665878


In [ ]:
class_labels = ["0","1"]

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Create a heatmap of the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Generate a classification report
report = classification_report(true_labels, predicted_labels, target_names=class_labels)

# Print the classification report
print(report)

In [ ]:
# # Create an array to represent time periods (e.g., days, months)
# time_periods = np.arange(len(true_labels))

# # Calculate accuracy at each time period
# accuracies = [np.mean(true_labels[:i+1] == predicted_labels[:i+1]) for i in range(len(true_labels))]

# # Create a line plot
# plt.figure(figsize=(10, 6))
# plt.plot(time_periods, accuracies)
# plt.xlabel('Time Period')
# plt.ylabel('Accuracy')
# plt.title('Accuracy Over Time')
# plt.grid(True)
# plt.show()


In [ ]:
# Compute ROC curve
fpr, tpr, _ = roc_curve(true_labels, probabilities[:, 1])  # Assuming binary classification

# Calculate AUC (Area Under the Curve)
auc = roc_auc_score(true_labels, probabilities[:, 1])

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {auc:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()


In [ ]:
# helper_model.print_wrong_classifications(predicted_labels, true_labels, raw_datasets['test'])

Use the saved model to predit on the unlabled testing data.

In [ ]:
# Define the path to the saved model directory
saved_model_dir = "saved_model"

# Load the saved tokenizer
tokenizer = BertTokenizer.from_pretrained(saved_model_dir)

# Load the saved model
model = BertForSequenceClassification.from_pretrained(saved_model_dir)
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
trainer = Trainer(model=model)
predictions = trainer.predict(tokenized_datasets["test_unlabeled"])
# Apply softmax to convert logits to probabilities
probabilities = softmax(predictions.predictions, axis=1)

# Get the predicted class labels
predicted_labels = np.argmax(probabilities, axis=1)

print("Probabilities:\n", probabilities)
print("Predicted Labels:\n", predicted_labels)

Probabilities:
 [[0.00167965 0.9983203 ]
 [0.01258648 0.9874135 ]
 [0.00146793 0.99853206]
 ...
 [0.8082649  0.1917351 ]
 [0.00115817 0.9988418 ]
 [0.00138278 0.9986172 ]]
Predicted Labels:
 [1 1 1 ... 0 1 1]


In [ ]:
result_df = pd.DataFrame({
    'text': raw_datasets['test_unlabeled']['text'],
    'Predicted_Labels': predicted_labels,
    'Probability_Class_0': probabilities[:, 0],
    'Probability_Class_1': probabilities[:, 1], # 2 classes
})

output_csv_path = "predictions.csv"
result_df.to_csv(output_csv_path, index=False)
print(f"Predictions have been saved to {output_csv_path}")

Predictions have been saved to predictions.csv
